# Projeto Covid-19
Projeto com intensão de analisar e apresentar os dados sobre a covid19 no ano de 2020.

In [ ]:
#importando as bibliotecas 
import pandas as pd
import numpy as np
import re
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
plt.style.use('seaborn')
#bibliotecas utilizadas na predição
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#fazer upload do arquivo para o ambiente COLAB
from google.colab import files
arq = files.upload()

In [ ]:
#Utilizando o 'parse_update' para já especificar as colunas que terão o fomato de data
df = pd.read_csv('covid_19_data.csv', parse_dates=['ObservationDate', 'Last Update'])
df.head()

In [ ]:
#verificando os tipos de cada coluna
df.dtypes

In [ ]:
#criando uma expressão regular utilizando o biblioteca 're' para corrigir os nomes das colunas
def corrige_colunas(col_name):
  return re.sub(r'[/| ]', '_', col_name).lower()

In [ ]:
#chamando a função para corrigir os nomes das colunas
df.columns = [corrige_colunas(col) for col in df.columns] #chamar a função para cada coluna contida no dataFrame
df.rename(columns={'observationdate': 'observation_date'}, inplace = True) #alterando coluna que não se encaixa no padrão da função
df.head()

## **Selecionando os dados brasileiros**

Criando um subset somente com os dados da Covid-19 no território brasileiro.

In [ ]:
#conhecendo os países contidos no dataframe
df.country_region.unique()

In [ ]:
# apresentando a filtragem por país
df.loc[df.country_region == 'Brazil']

In [ ]:
#selecionando os dados filtrados por pais e qtde de casos
brasil = df.loc[
    (df.country_region == 'Brazil') & 
    (df.confirmed > 0)
]
brasil.head()

### **Analisando dados confirmados**


In [ ]:
#apresentando a evolução dos casos confirmados em um gráfico de linha
px.line(brasil, 'observation_date', 'confirmed', title='Casos confirmados no Brasil')

### **Calculando os casos por dia**

Calculando o crescimento de casos diarios atrvés de programação funcional.

- Obs.: loc localiza os objetos pelo nome.
- Obs.2: iloc localiza os objetos pelo índice.

In [ ]:
brasil['novos_casos'] = list(map(
    lambda x: 0 if (x == 0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
px.line(brasil, x='observation_date', y='novos_casos', title='Novos casos por Dia')

### **Calculado a quantidade de mortes**

In [ ]:
#plotando os numero de mortes com gráfico visial
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=brasil.observation_date, 
        y=brasil.deaths, 
        name='Mortes', 
        mode='lines+markers', 
        line={'color':'red'}
    )
)

#modificando layout do grafico
fig.update_layout(title='Mortes por Covid-19 no Brasil')
fig.show()

### **Calculando taxa de crescimento**

In [ ]:
#taxa de crescimento geral de todo o periodo
def taxa_crescimento(dataframe, variavel, data_inicio=None, data_fim=None):
    #definindo o valores de inicio das datas
    if data_inicio == None:
        data_inicio = dataframe.observation_date.loc[dataframe[variavel] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = dataframe.observation_date.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
    
    #definindo os valores de passado e presente
    passado = dataframe.loc[dataframe.observation_date == data_inicio, variavel].values[0]
    presente = dataframe.loc[dataframe.observation_date == data_fim, variavel].values[0]
    
    #definindo o numero de pontos temporais a serem avaliados
    n = (data_fim - data_inicio).days
    
    #calculando a taxa de crescimento
    taxa = (presente/passado)**(1/n) - 1

    return taxa*100

#apresentando a taxa de crescimento médio do Covid-19 no Brasil
taxa_crescimento(brasil, 'confirmed')

In [ ]:
#calculando a taxa de crescimento diário
def taxa_crescimento_diaria(dataframe, variavel, data_inicio=None):
  #definindo o valores de inicio das datas
    if data_inicio == None:
        data_inicio = dataframe.observation_date.loc[dataframe[variavel] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
    
    data_fim = dataframe.observation_date.max()

    #definindo o numero de pontos temporais a serem avaliados
    n = (data_fim - data_inicio).days

    #calculando a taxa diaria
    #prepresentação da função: (hoje - ontem) / ontem
    taxas = list(map(
        lambda x: (dataframe[variavel].iloc[x] - dataframe[variavel].iloc[x-1]) / dataframe[variavel].iloc[x-1],
        range(1,n+1)
    ))

    return np.array(taxas) * 100

taxa_diaria = taxa_crescimento_diaria(brasil, 'confirmed')
taxa_diaria

In [ ]:
#plotando dados graficamente
primeiro_dia = brasil.observation_date.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observation_date.max())[1:],
        y=taxa_diaria,
        title='Taxa de crescimento de casos confirmados no Brasil')

### **Modelos de Predições**

In [ ]:
#criando um novo dataframe somente com os casos confirmados e definindo a data com indice
casos_confirmados = brasil.confirmed
casos_confirmados.index = brasil.observation_date

#fazendo a deconposição dos dados confirmados
res = seasonal_decompose(casos_confirmados)

In [ ]:
#plotando os valores em 4 gráficos unidos em uma única figura
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,8))

ax1.plot(res.observed) #valores observados
ax2.plot(res.trend) #tedência
ax3.plot(res.seasonal) #sazonalidade
ax4.plot(casos_confirmados.index, res.resid) #ruído
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

## **Modelando os dados utilizando a biblioteca ARIMA**

Aplicando a metodologia de modelar o futuro como função do passado.


In [ ]:
#instalando a biblioteca, pois a mesma não consta na base do COLAB
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
modelo = auto_arima(casos_confirmados)

In [ ]:
#apresentando os dados modelados com a melhor esturura do arima
fig = go.Figure(go.Scatter(
    x=casos_confirmados.index,
    y=casos_confirmados,
    name='Observados'
))

fig.add_trace(go.Scatter(
    x=casos_confirmados.index,
    y=modelo.predict_in_sample(),
    name='Preditos'
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20'),
    y=modelo.predict(31),
    name='Forecast'
))

fig.update_layout(title='Previsaão de casos confirmados nos Brasil para os próximos 30 dias.')
fig.show()

## **Modelo de crescimento com a biblioteca fbpropht**
Metodologia do modelo de crescimento que visa prever o ponto de virada.
- é necessário passar por um pré processamento nos dados antes de se utilizar.

In [ ]:
#importando a biblioteco do anaconda
!conda install -c conda-forge fbpropht -y

In [ ]:
from fbprophet import Prophet

In [ ]:
# pré processamentos

treinamento = casos_confirmados.reset_index()[:-5] #base para treinar o modelo
teste = casos_confirmados.reset_index()[-5:] #pegando os ultimos dados para teste

#renomeando as colunas:
treinamento.rename(columns={'observation_date':'ds', 'confirmed':'y'}, inplace=True)
teste.rename(columns={'observation_date':'ds', 'confirmed':'y'}, inplace=True)

#definindo modelo de crescimento:
profeta = Prophet(growth='logistic', changepoints=['2020-03-21', '2020-03-30', '2020-04-25', '2020-05-03', '2020-05-10'])

#montando um cenário com ponto máximo de toda a população sendo contaminada:
populacao = 211463256
treinamento['cap'] = populacao

#treinando o modelo:
profeta.fit(treinamento)

#construindo as previsões com base no cenário:
# Construindo previsões para o futuro
future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = populacao
forecast =  profeta.predict(future_dates)


In [ ]:
#apresentação da análise no cenário escolhido
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=treinamento.index, y=treinamento, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=treinamento.ds, y=treinamento.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()